<a href="https://colab.research.google.com/github/simulate111/General/blob/main/T%C3%B3rshavn_Faroe_Islands.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [14]:
import requests
import pandas as pd
import time

# --- CONFIGURATION ---
# NEW ENDPOINT (No API Key required)
base_url = "https://opendataapi.dmi.dk/v2/metObs/collections/observation/items"

start_date = "2024-01-01T00:00:00Z"
end_date = "2025-01-01T00:00:00Z"
limit = 300000

# PRIORITY LIST OF STATIONS
# 1. Tórshavn (06011) - Primary, but often managed by local FMO (might be empty here)
# 2. Vágar Airport (06010) - Reliable fallback
# 3. Akraberg (06009) - Southern lighthouse
station_candidates = ["06011", "06010", "06009"]

# Parameters to fetch
tasks = [
    {"param": "temp_dry",   "name": "Temperature_C"},
    {"param": "wind_speed", "name": "Wind_Speed_ms"}
]

active_station = None
dfs = []

print("--- Starting Data Retrieval for Faroe Islands (2024) ---")
print("Using new DMI Open Data Endpoint (No Authentication Required)")

# Step 1: Find a working station
for station_id in station_candidates:
    print(f"\nChecking Station {station_id}...")

    # Test fetch to see if data exists
    test_params = {
        "stationId": station_id,
        "datetime": f"{start_date}/{end_date}",
        "parameterId": "temp_dry",
        "limit": 10
    }

    try:
        r = requests.get(base_url, params=test_params)

        if r.status_code == 200:
            data = r.json().get('features', [])
            if len(data) > 0:
                print(f" > SUCCESS: Station {station_id} is active and has data.")
                active_station = station_id
                break
            else:
                print(f" > Station {station_id} returned 0 rows. Trying next...")
        else:
            print(f" > Error checking station {station_id}: {r.status_code}")
    except Exception as e:
        print(f" > Connection failed for {station_id}: {e}")

if not active_station:
    print("\nCRITICAL ERROR: No active stations found with data for 2024.")
    exit()

# Step 2: Fetch full data for the active station
print(f"\nFetching full 2024 datasets from Active Station: {active_station}...")

for task in tasks:
    print(f" > Requesting {task['name']}...")

    params = {
        "stationId": active_station,
        "datetime": f"{start_date}/{end_date}",
        "parameterId": task['param'],
        "limit": limit
    }

    try:
        r = requests.get(base_url, params=params)

        if r.status_code == 200:
            data = r.json().get('features', [])
            records = []

            for item in data:
                records.append({
                    'Time': item['properties']['observed'],
                    task['name']: item['properties']['value']
                })

            df_temp = pd.DataFrame(records)

            if not df_temp.empty:
                df_temp['Time'] = pd.to_datetime(df_temp['Time'])

                # Resample to Hourly Averages
                df_temp = df_temp.set_index('Time').resample('h').mean()

                dfs.append(df_temp)
                print(f"   > Got {len(df_temp)} hourly records for {task['name']}.")
            else:
                print(f"   > WARNING: No data found for {task['name']}.")
        else:
            print(f"   > API Error: {r.status_code}")

    except Exception as e:
        print(f"   > Error: {e}")

    time.sleep(1)

# Step 3: Merge and Save
if dfs:
    print("\nMerging and formatting...")
    df_final = pd.concat(dfs, axis=1)
    df_final = df_final.sort_index()

    # Fill small gaps
    df_final = df_final.interpolate(method='linear', limit=2)
    df_final = df_final.dropna(how='all')

    # Format columns
    df_final = df_final.reset_index()
    df_final['Date'] = df_final['Time'].dt.strftime('%Y-%m-%d')
    df_final['Hour'] = df_final['Time'].dt.strftime('%H:%M')

    cols = ['Date', 'Hour', 'Temperature_C', 'Wind_Speed_ms']
    df_final = df_final[[c for c in cols if c in df_final.columns]]

    filename = f"Faroe_Station_{active_station}_Wind_Temp_2024.csv"
    df_final.to_csv(filename, index=False)

    print("-" * 30)
    print(df_final.head())
    print("-" * 30)
    print(f"Success! Data saved to: {filename}")
else:
    print("Failed to compile dataset.")

--- Starting Data Retrieval for Faroe Islands (2024) ---
Using new DMI Open Data Endpoint (No Authentication Required)

Checking Station 06011...
 > SUCCESS: Station 06011 is active and has data.

Fetching full 2024 datasets from Active Station: 06011...
 > Requesting Temperature_C...
   > Got 4866 hourly records for Temperature_C.
 > Requesting Wind_Speed_ms...
   > Got 4866 hourly records for Wind_Speed_ms.

Merging and formatting...
------------------------------
         Date   Hour  Temperature_C  Wind_Speed_ms
0  2024-06-12  07:00       9.500000       3.100000
1  2024-06-12  08:00       9.916667       3.916667
2  2024-06-12  09:00      10.233333       4.766667
3  2024-06-12  10:00      11.166667       4.750000
4  2024-06-12  11:00      11.983333       3.050000
------------------------------
Success! Data saved to: Faroe_Station_06011_Wind_Temp_2024.csv
